# 05: Delaunay Triangulation and Voronoi Diagrams

*Authors: Lars Nitzschke, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Data Structures    
    3.1. Slab Decomposition  
    3.2. Vertical Decomposition
4. References  

## 1. Introduction

TODO: Introduction (Def., as statet in the lecture; explanation of the examples below)

TODO: Images

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [ ]:
# Python standard library imports
from abc import ABC, abstractmethod
from typing import Any, Optional, Iterable
import random

# Data structure, geometry and visualisation module imports
from modules.data_structures import Comparator, ComparisonResult as CR, Vertex, HalfEdge, DoublyConnectedEdgeList, PointLocation, Face, VDLineSegment, AnimationBinaryTreeDict, PLSearchStructure
from modules.geometry import Point, PointReference, PointSequence, LineSegment, EPSILON, Rectangle, VerticalOrientation as VORT, HorizontalOrientation as HORT
from modules.visualisation import VisualisationTool, PointLocationInstance, VerticalExtensionMode, PointLocationMode, PointSetInstance

Additionally, we create an object for our visualisation tool and register a few example instances.

In [ ]:
visualisation_tool = VisualisationTool(400, 400, PointSetInstance())
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

c = 0.5 * canvas_size
r = 0.9 * c
s = c - r
t = c + r
u = (t - s) / 36

points = [Point(s + 20*u, t -  1*u), Point(s + 25*u, t -  6*u), Point(s + 31*u, t -  9*u), Point(s + 28*u, t - 11*u),
          Point(s + 19*u, t - 14*u), Point(s + 30*u, t - 21*u), Point(s + 27*u, t - 29*u), Point(s + 22*u, t - 33*u),
          Point(s + 13*u, t - 31*u), Point(s +  8*u, t - 34*u), Point(s +  2*u, t - 30*u), Point(s + 11*u, t - 24*u),
          Point(s +  5*u, t - 22*u), Point(s +  1*u, t - 17*u), Point(s +  4*u, t - 14*u), Point(s +  6*u, t -  6*u),
          Point(s + 10*u, t -  3*u), Point(s + 15*u, t - 17*u), Point(s + 18*u, t - 27*u)]

visualisation_tool.register_example_instance("Example Point Set", points)

## 3. Algorithms

### 3.1 Delaunay Triangulation

In [ ]:
visualisation_tool.display()

In [ ]:
# Construction of the Delaunay Triangulation
class DelaunayTriangulation(DoublyConnectedEdgeList):
    def __init__(self, points: Iterable[Point] = [], edges: Iterable[tuple[int, int]] = []):
        super().__init__(points, edges)

    def insert(self, point: Point) -> None:
        containing_triangle = self.find_containing_face(point)  # TODO: Replace by search structure
        # search returns triangle if point is inside and edge if point is on it.
        on_edge: HalfEdge = None
        inside: bool = True
        if inside:
            vertex = super().add_vertex(point)  # TODO: Maybe with less checks that the point is correct?
            # Case inside triangle:
            surrounding_vertices = list(containing_triangle.outer_vertices())
            if len(surrounding_vertices != 3):
                raise Exception("Face is not a triangle")
            self._add_edge(vertex, surrounding_vertices[0])
            self._add_edge(vertex, surrounding_vertices[1])
            self._add_edge(vertex, surrounding_vertices[2])
            #self.legalize_edge(vertex, 0)
            #self.legalize_edge(vertex, 1)
            #self.legalize_edge(vertex, 2)
            # TODO: Update Search Structure: split
        else:
            vertex = super().add_vertex_in_edge(on_edge, point)
            # TODO: Case point on edge 
            #self.add_edge_by_points(point, p_l)  # points p_l, p_k from edge that point is on
            #self.add_edge_by_points(point, p_k)
            #self.legalize_edge(point, 0)
            #self.legalize_edge(point, 1)
            #self.legalize_edge(point, 2)
            #self.legalize_edge(point, 3)
            pass

    def legalize_edge(self, vertex: Vertex, edge: HalfEdge) -> None:
        if self.is_edge_illegal(edge):  # Edge flip
            opposing_vertex: Vertex = edge.twin.next.destination
            # TODO: Remove "edge"
            self._add_edge(vertex, opposing_vertex.point)  # Add flipped edge
            # TODO: Update Search Structure: flip

            self.legalize_edge(vertex, edge.twin.next)
            self.legalize_edge(vertex, edge.twin.prev)

    def is_edge_illegal(self, edge) -> bool: # TODO
        # TODO: Use description in [1, section 9.3] at the end for points p_0, p_-1, p_-2 (Check these special points by coordinates =abs(3M))
        # Otherwise: Lemma 9.4 [1] / Lemma 1 [lecture 05] circumcircle C around triangle <=> illegal if inside C
        return True

random_seed = 42
def delaunay_triangulation(points: set[Point]) -> PointSequence:
    points: list[Point] = list(points)
    if random_seed is not None:
        random.seed(random_seed)
    random.shuffle(points)  # Randomized Incremental Construction
    triangulation: DelaunayTriangulation = DelaunayTriangulation()

    # TODO: initialize triangulation with points p_0 = (3M, 0), p_-1 = (0, 3M), p_-2 = (-3M, -3M)
    # where M is the max abs value of any coordinate of any point

    # TODO: initialize search structure with initial triangle p_0p_-1p_-2

    for point in points:
        triangulation.insert(point)
